# Step 1: Linear Algebra - Interactive Learning

This notebook provides hands-on exploration of linear algebra concepts fundamental to machine learning.

## 🎯 Learning Objectives
- Visualize vector operations and matrix transformations
- Understand eigenvalues and eigenvectors geometrically
- Implement and understand PCA
- See how linear algebra powers neural networks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs, load_iris
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
np.random.seed(42)

## 1. Vector Operations and Geometry

Let's start by visualizing basic vector operations.

In [ ]:
def plot_vectors_2d(vectors, labels=None, colors=None, title="Vector Operations"):
    """Plot 2D vectors from origin."""
    fig, ax = plt.subplots(figsize=(8, 8))
    
    if colors is None:
        colors = ['red', 'blue', 'green', 'purple', 'orange']
    
    if labels is None:
        labels = [f'v{i+1}' for i in range(len(vectors))]
    
    for i, (vec, label, color) in enumerate(zip(vectors, labels, colors)):
        ax.quiver(0, 0, vec[0], vec[1], 
                 angles='xy', scale_units='xy', scale=1,
                 color=color, width=0.006, label=label)
        
        # Add vector endpoint
        ax.plot(vec[0], vec[1], 'o', color=color, markersize=8)
        
        # Add text label
        ax.text(vec[0]*1.1, vec[1]*1.1, label, fontsize=12, color=color)
    
    # Set equal aspect ratio and grid
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    
    # Set limits
    all_coords = np.array(vectors).flatten()
    max_coord = np.max(np.abs(all_coords)) * 1.2
    ax.set_xlim(-max_coord, max_coord)
    ax.set_ylim(-max_coord, max_coord)
    
    ax.set_title(title)
    ax.legend()
    plt.show()

# Example vectors
u = np.array([3, 2])
v = np.array([1, 4])
u_plus_v = u + v
u_minus_v = u - v

print(f"u = {u}")
print(f"v = {v}")
print(f"u + v = {u_plus_v}")
print(f"u - v = {u_minus_v}")

plot_vectors_2d([u, v, u_plus_v, u_minus_v], 
                ['u', 'v', 'u+v', 'u-v'],
                ['red', 'blue', 'green', 'purple'],
                'Vector Addition and Subtraction')

### Interactive: Dot Product and Angle

In [ ]:
def analyze_dot_product(u, v):
    """Analyze dot product and angle between vectors."""
    dot_product = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    
    cos_theta = dot_product / (norm_u * norm_v)
    theta_rad = np.arccos(np.clip(cos_theta, -1, 1))  # Clip to handle numerical errors
    theta_deg = np.degrees(theta_rad)
    
    print(f"Vector u: {u}")
    print(f"Vector v: {v}")
    print(f"||u|| = {norm_u:.3f}")
    print(f"||v|| = {norm_v:.3f}")
    print(f"u · v = {dot_product:.3f}")
    print(f"cos(θ) = {cos_theta:.3f}")
    print(f"θ = {theta_deg:.1f}°")
    
    if abs(cos_theta) < 1e-10:
        print("Vectors are ORTHOGONAL (perpendicular)")
    elif cos_theta > 0:
        print("Vectors point in SIMILAR directions")
    else:
        print("Vectors point in OPPOSITE directions")
    
    # Visualize
    plot_vectors_2d([u, v], ['u', 'v'], ['red', 'blue'], 
                   f'Dot Product = {dot_product:.2f}, Angle = {theta_deg:.1f}°')

# Test different vector pairs
print("=== Example 1: Acute angle ===")
analyze_dot_product(np.array([3, 2]), np.array([2, 3]))

print("\n=== Example 2: Obtuse angle ===")
analyze_dot_product(np.array([3, 2]), np.array([-1, 3]))

print("\n=== Example 3: Orthogonal vectors ===")
analyze_dot_product(np.array([1, 0]), np.array([0, 1]))

## 2. Matrix Transformations

Matrices represent linear transformations. Let's visualize how they transform shapes.

In [ ]:
def visualize_transformation(matrix, title="Matrix Transformation"):
    """Visualize how a matrix transforms a unit square and circle."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original shapes
    # Unit square vertices
    square = np.array([[0, 1, 1, 0, 0],
                       [0, 0, 1, 1, 0]])
    
    # Unit circle
    theta = np.linspace(0, 2*np.pi, 100)
    circle = np.array([np.cos(theta), np.sin(theta)])
    
    # Grid lines
    grid_x = np.array([[-2, 2], [-2, 2], [-1, 1], [-1, 1], [0, 0], [0, 0]])
    grid_y = np.array([[-1, -1], [1, 1], [-2, 2], [-2, 2], [-2, 2], [-2, 2]])
    
    # Transform shapes
    square_transformed = matrix @ square
    circle_transformed = matrix @ circle
    grid_transformed = matrix @ np.vstack([grid_x, grid_y])
    
    # Plot original
    axes[0].plot(square[0], square[1], 'b-', linewidth=2, label='Unit Square')
    axes[0].plot(circle[0], circle[1], 'r-', linewidth=2, label='Unit Circle')
    for i in range(0, len(grid_x), 2):
        axes[0].plot(grid_x[i], grid_y[i], 'k-', alpha=0.3)
    axes[0].set_aspect('equal')
    axes[0].grid(True, alpha=0.3)
    axes[0].set_title('Original')
    axes[0].legend()
    axes[0].set_xlim(-3, 3)
    axes[0].set_ylim(-3, 3)
    
    # Plot transformed
    axes[1].plot(square_transformed[0], square_transformed[1], 'b-', linewidth=2, label='Transformed Square')
    axes[1].plot(circle_transformed[0], circle_transformed[1], 'r-', linewidth=2, label='Transformed Circle')
    for i in range(0, grid_transformed.shape[1], 2):
        axes[1].plot(grid_transformed[0, i:i+2], grid_transformed[1, i:i+2], 'k-', alpha=0.3)
    axes[1].set_aspect('equal')
    axes[1].grid(True, alpha=0.3)
    axes[1].set_title('Transformed')
    axes[1].legend()
    
    # Auto-scale for transformed plot
    all_coords = np.concatenate([square_transformed.flatten(), circle_transformed.flatten()])
    max_coord = np.max(np.abs(all_coords)) * 1.1
    axes[1].set_xlim(-max_coord, max_coord)
    axes[1].set_ylim(-max_coord, max_coord)
    
    # Plot both overlaid
    axes[2].plot(square[0], square[1], 'b--', alpha=0.5, linewidth=2, label='Original')
    axes[2].plot(circle[0], circle[1], 'r--', alpha=0.5, linewidth=2)
    axes[2].plot(square_transformed[0], square_transformed[1], 'b-', linewidth=2, label='Transformed')
    axes[2].plot(circle_transformed[0], circle_transformed[1], 'r-', linewidth=2)
    axes[2].set_aspect('equal')
    axes[2].grid(True, alpha=0.3)
    axes[2].set_title('Overlay')
    axes[2].legend()
    axes[2].set_xlim(-max_coord, max_coord)
    axes[2].set_ylim(-max_coord, max_coord)
    
    plt.suptitle(f'{title}\nMatrix: {matrix}')
    plt.tight_layout()
    plt.show()
    
    # Print matrix properties
    det = np.linalg.det(matrix)
    print(f"Determinant: {det:.3f}")
    if abs(det) < 1e-10:
        print("Matrix is SINGULAR (not invertible)")
    elif det < 0:
        print("Matrix FLIPS orientation (negative determinant)")
    else:
        print("Matrix PRESERVES orientation (positive determinant)")
    print(f"Area scaling factor: {abs(det):.3f}")

# Test different transformations
print("=== Scaling Matrix ===")
scaling = np.array([[2, 0],
                    [0, 0.5]])
visualize_transformation(scaling, "Scaling Transformation")

In [ ]:
print("\n=== Rotation Matrix ===")
angle = np.pi/4  # 45 degrees
rotation = np.array([[np.cos(angle), -np.sin(angle)],
                     [np.sin(angle), np.cos(angle)]])
visualize_transformation(rotation, "45° Rotation")

print("\n=== Shear Matrix ===")
shear = np.array([[1, 1],
                  [0, 1]])
visualize_transformation(shear, "Horizontal Shear")

print("\n=== Reflection Matrix ===")
reflection = np.array([[1, 0],
                       [0, -1]])
visualize_transformation(reflection, "Reflection across X-axis")

## 3. Eigenvalues and Eigenvectors

Eigenvectors are special directions that matrices only stretch (don't rotate).

In [ ]:
def visualize_eigenvectors(matrix, title="Eigenanalysis"):
    """Visualize eigenvectors and their effect."""
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Plot 1: Original vectors and eigenvectors
    colors = ['red', 'blue']
    for i, (val, vec, color) in enumerate(zip(eigenvalues, eigenvectors.T, colors)):
        if np.isreal(val) and np.isreal(vec).all():
            vec = np.real(vec)
            val = np.real(val)
            
            # Plot eigenvector
            axes[0].quiver(0, 0, vec[0], vec[1], 
                          angles='xy', scale_units='xy', scale=1,
                          color=color, width=0.008, 
                          label=f'λ={val:.2f}')
            
            # Plot the line through eigenvector
            t = np.linspace(-3, 3, 100)
            line_x = t * vec[0]
            line_y = t * vec[1]
            axes[0].plot(line_x, line_y, '--', color=color, alpha=0.5)
    
    axes[0].set_aspect('equal')
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xlim(-3, 3)
    axes[0].set_ylim(-3, 3)
    axes[0].set_title('Eigenvectors')
    axes[0].legend()
    
    # Plot 2: Unit circle transformation
    theta = np.linspace(0, 2*np.pi, 100)
    circle = np.array([np.cos(theta), np.sin(theta)])
    circle_transformed = matrix @ circle
    
    axes[1].plot(circle[0], circle[1], 'b--', alpha=0.5, label='Unit Circle')
    axes[1].plot(circle_transformed[0], circle_transformed[1], 'r-', linewidth=2, label='Transformed')
    
    # Add eigenvector directions
    for i, (val, vec, color) in enumerate(zip(eigenvalues, eigenvectors.T, colors)):
        if np.isreal(val) and np.isreal(vec).all():
            vec = np.real(vec)
            val = np.real(val)
            
            # Show how eigenvector gets stretched
            original_vec = vec
            transformed_vec = matrix @ vec
            
            axes[1].arrow(0, 0, original_vec[0], original_vec[1],
                         head_width=0.1, head_length=0.1, fc=color, ec=color,
                         alpha=0.5, width=0.02)
            axes[1].arrow(0, 0, transformed_vec[0], transformed_vec[1],
                         head_width=0.1, head_length=0.1, fc=color, ec=color,
                         width=0.03)
    
    axes[1].set_aspect('equal')
    axes[1].grid(True, alpha=0.3)
    axes[1].legend()
    axes[1].set_title('Circle → Ellipse')
    
    # Auto-scale
    max_coord = np.max(np.abs(circle_transformed)) * 1.1
    axes[1].set_xlim(-max_coord, max_coord)
    axes[1].set_ylim(-max_coord, max_coord)
    
    # Plot 3: Eigenvalue spectrum
    real_eigenvalues = np.real(eigenvalues)
    axes[2].bar(range(len(real_eigenvalues)), real_eigenvalues, color=colors[:len(real_eigenvalues)])
    axes[2].set_xlabel('Eigenvalue Index')
    axes[2].set_ylabel('Eigenvalue')
    axes[2].set_title('Eigenvalue Spectrum')
    axes[2].grid(True, alpha=0.3)
    
    plt.suptitle(f'{title}\nMatrix: {matrix}')
    plt.tight_layout()
    plt.show()
    
    # Print detailed analysis
    print(f"Eigenvalues: {eigenvalues}")
    print(f"Eigenvectors (columns):")
    print(eigenvectors)
    
    # Verify Av = λv
    print("\nVerification (Av = λv):")
    for i, (val, vec) in enumerate(zip(eigenvalues, eigenvectors.T)):
        if np.isreal(val) and np.isreal(vec).all():
            Av = matrix @ vec
            lambda_v = val * vec
            error = np.linalg.norm(Av - lambda_v)
            print(f"Eigenvector {i+1}: ||Av - λv|| = {error:.2e}")

# Test with different matrices
print("=== Symmetric Matrix (Real Eigenvalues) ===")
symmetric = np.array([[3, 1],
                      [1, 3]])
visualize_eigenvectors(symmetric, "Symmetric Matrix")

In [ ]:
print("\n=== Covariance-like Matrix ===")
cov_matrix = np.array([[2, 1.5],
                       [1.5, 1]])
visualize_eigenvectors(cov_matrix, "Covariance-like Matrix")

print("\n=== Non-symmetric Matrix ===")
nonsymmetric = np.array([[2, 3],
                         [1, 0]])
visualize_eigenvectors(nonsymmetric, "Non-symmetric Matrix")

## 4. Principal Component Analysis (PCA)

PCA finds the principal directions of variance in data using eigenanalysis.

In [ ]:
# Load our custom PCA implementation
exec(open('../code/01_pca.py').read())

# Generate sample data
np.random.seed(42)
n_samples = 200

# Create correlated data
mean = [2, 3]
cov = [[2, 1.5], [1.5, 1.5]]
data = np.random.multivariate_normal(mean, cov, n_samples)

# Apply PCA
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data)

# Interactive visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Original data with principal components
axes[0, 0].scatter(data[:, 0], data[:, 1], alpha=0.6, s=30)

# Draw principal components
mean_point = pca.mean_
colors = ['red', 'blue']
for i, (component, variance, color) in enumerate(zip(pca.components_, pca.explained_variance_, colors)):
    # Scale by square root of eigenvalue
    scale = 2 * np.sqrt(variance)
    axes[0, 0].arrow(mean_point[0], mean_point[1], 
                     scale * component[0], scale * component[1],
                     head_width=0.1, head_length=0.1, 
                     fc=color, ec=color, width=0.03,
                     label=f'PC{i+1} (λ={variance:.2f})')

axes[0, 0].set_title('Original Data with Principal Components')
axes[0, 0].set_xlabel('Feature 1')
axes[0, 0].set_ylabel('Feature 2')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axis('equal')

# Plot 2: Data in PC space
axes[0, 1].scatter(data_pca[:, 0], data_pca[:, 1], alpha=0.6, s=30)
axes[0, 1].set_title('Data in Principal Component Space')
axes[0, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
axes[0, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axis('equal')

# Plot 3: Explained variance
axes[1, 0].bar(['PC1', 'PC2'], pca.explained_variance_ratio_, color=colors)
axes[1, 0].set_title('Explained Variance Ratio')
axes[1, 0].set_ylabel('Variance Ratio')
axes[1, 0].grid(True, alpha=0.3)

# Add percentage labels
for i, ratio in enumerate(pca.explained_variance_ratio_):
    axes[1, 0].text(i, ratio + 0.01, f'{ratio:.1%}', ha='center', va='bottom')

# Plot 4: 1D projection (dimensionality reduction)
pca_1d = PCA(n_components=1)
data_1d = pca_1d.fit_transform(data)
data_reconstructed = pca_1d.inverse_transform(data_1d)

axes[1, 1].scatter(data[:, 0], data[:, 1], alpha=0.4, s=20, label='Original')
axes[1, 1].scatter(data_reconstructed[:, 0], data_reconstructed[:, 1], 
                   alpha=0.8, s=20, label='1D Projection')

# Draw projection lines for some points
for i in range(0, len(data), 20):
    axes[1, 1].plot([data[i, 0], data_reconstructed[i, 0]], 
                    [data[i, 1], data_reconstructed[i, 1]], 
                    'k-', alpha=0.3, linewidth=0.5)

axes[1, 1].set_title(f'1D Reconstruction ({pca_1d.explained_variance_ratio_[0]:.1%} variance)')
axes[1, 1].set_xlabel('Feature 1')
axes[1, 1].set_ylabel('Feature 2')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axis('equal')

plt.tight_layout()
plt.show()

print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Cumulative variance: {np.cumsum(pca.explained_variance_ratio_)}")
print(f"Principal components (rows):")
print(pca.components_)

### PCA on Real Data: Iris Dataset

In [ ]:
# Load and explore Iris dataset
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

print(f"Dataset shape: {X.shape}")
print(f"Features: {feature_names}")
print(f"Classes: {target_names}")

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca_iris = PCA()
X_pca = pca_iris.fit_transform(X_scaled)

# Create interactive 3D plot
fig = go.Figure()

colors = ['red', 'green', 'blue']
for i, (target, color, name) in enumerate(zip(np.unique(y), colors, target_names)):
    mask = y == target
    fig.add_trace(go.Scatter3d(
        x=X_pca[mask, 0],
        y=X_pca[mask, 1], 
        z=X_pca[mask, 2],
        mode='markers',
        marker=dict(size=5, color=color),
        name=name,
        text=[f'{name}<br>PC1: {x:.2f}<br>PC2: {y:.2f}<br>PC3: {z:.2f}' 
              for x, y, z in zip(X_pca[mask, 0], X_pca[mask, 1], X_pca[mask, 2])],
        hovertemplate='%{text}<extra></extra>'
    ))

fig.update_layout(
    title='Iris Dataset in 3D PCA Space',
    scene=dict(
        xaxis_title=f'PC1 ({pca_iris.explained_variance_ratio_[0]:.1%} variance)',
        yaxis_title=f'PC2 ({pca_iris.explained_variance_ratio_[1]:.1%} variance)',
        zaxis_title=f'PC3 ({pca_iris.explained_variance_ratio_[2]:.1%} variance)'
    )
)

fig.show()

# Summary statistics
print(f"\nExplained variance by component:")
for i, ratio in enumerate(pca_iris.explained_variance_ratio_):
    print(f"  PC{i+1}: {ratio:.3f} ({ratio*100:.1f}%)")

print(f"\nCumulative explained variance:")
cumsum = np.cumsum(pca_iris.explained_variance_ratio_)
for i, cum_ratio in enumerate(cumsum):
    print(f"  PC1-{i+1}: {cum_ratio:.3f} ({cum_ratio*100:.1f}%)")

## 5. Linear Regression with Normal Equation

Let's implement linear regression using the closed-form solution.

In [ ]:
# Load our linear regression implementation
exec(open('../code/01_linear_regression.py').read())

# Generate sample data
np.random.seed(42)
n_samples = 100
X = np.random.randn(n_samples, 2)
true_weights = np.array([3.0, -2.0])
true_intercept = 1.5
noise = 0.3 * np.random.randn(n_samples)
y = X @ true_weights + true_intercept + noise

print(f"True weights: {true_weights}")
print(f"True intercept: {true_intercept}")
print(f"Noise std: 0.3")

# Fit model
model = LinearRegressionNormalEquation()
model.fit(X, y)

print(f"\nFitted weights: {model.weights}")
print(f"Fitted intercept: {model.intercept:.3f}")
print(f"R² score: {model.score(X, y):.3f}")

# Visualize results
y_pred = model.predict(X)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Actual vs Predicted
axes[0, 0].scatter(y, y_pred, alpha=0.6)
min_val, max_val = min(y.min(), y_pred.min()), max(y.max(), y_pred.max())
axes[0, 0].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
axes[0, 0].set_xlabel('Actual')
axes[0, 0].set_ylabel('Predicted')
axes[0, 0].set_title(f'Actual vs Predicted (R² = {model.score(X, y):.3f})')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Feature 1 relationship
axes[0, 1].scatter(X[:, 0], y, alpha=0.6, label='Data')
x1_range = np.linspace(X[:, 0].min(), X[:, 0].max(), 100)
x2_mean = np.mean(X[:, 1])
X_viz = np.column_stack([x1_range, np.full_like(x1_range, x2_mean)])
y_viz = model.predict(X_viz)
axes[0, 1].plot(x1_range, y_viz, 'r-', lw=2, label='Fitted line')
axes[0, 1].set_xlabel('Feature 1')
axes[0, 1].set_ylabel('Target')
axes[0, 1].set_title('Feature 1 vs Target (Feature 2 at mean)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Feature 2 relationship
axes[1, 0].scatter(X[:, 1], y, alpha=0.6, label='Data')
x2_range = np.linspace(X[:, 1].min(), X[:, 1].max(), 100)
x1_mean = np.mean(X[:, 0])
X_viz2 = np.column_stack([np.full_like(x2_range, x1_mean), x2_range])
y_viz2 = model.predict(X_viz2)
axes[1, 0].plot(x2_range, y_viz2, 'r-', lw=2, label='Fitted line')
axes[1, 0].set_xlabel('Feature 2')
axes[1, 0].set_ylabel('Target')
axes[1, 0].set_title('Feature 2 vs Target (Feature 1 at mean)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Residuals
residuals = y - y_pred
axes[1, 1].scatter(y_pred, residuals, alpha=0.6)
axes[1, 1].axhline(y=0, color='r', linestyle='--')
axes[1, 1].set_xlabel('Predicted')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].set_title('Residual Plot')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show the normal equation calculation step by step
print("\n=== Normal Equation Steps ===")
X_with_bias = np.column_stack([np.ones(X.shape[0]), X])
print(f"1. X with bias shape: {X_with_bias.shape}")

XtX = X_with_bias.T @ X_with_bias
print(f"2. X^T X shape: {XtX.shape}")
print(f"   X^T X condition number: {np.linalg.cond(XtX):.2e}")

Xty = X_with_bias.T @ y
print(f"3. X^T y shape: {Xty.shape}")

beta = np.linalg.solve(XtX, Xty)  # More stable than computing inverse
print(f"4. β = (X^T X)^(-1) X^T y: {beta}")
print(f"   Intercept: {beta[0]:.3f}")
print(f"   Weights: {beta[1:]}")

## 6. Neural Networks as Matrix Operations

Let's see how neural networks are fundamentally just sequences of matrix operations.

In [ ]:
def sigmoid(x):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))  # Clip to prevent overflow

def relu(x):
    """ReLU activation function."""
    return np.maximum(0, x)

class SimpleNeuralNetwork:
    """A simple 2-layer neural network to demonstrate matrix operations."""
    
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights randomly
        self.W1 = np.random.randn(input_size, hidden_size) * 0.5
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.5
        self.b2 = np.zeros((1, output_size))
        
    def forward(self, X, verbose=False):
        """Forward pass through the network."""
        if verbose:
            print(f"Input X shape: {X.shape}")
            print(f"W1 shape: {self.W1.shape}, b1 shape: {self.b1.shape}")
        
        # Layer 1: X @ W1 + b1
        self.z1 = X @ self.W1 + self.b1
        self.a1 = relu(self.z1)
        
        if verbose:
            print(f"z1 = X @ W1 + b1, shape: {self.z1.shape}")
            print(f"a1 = ReLU(z1), shape: {self.a1.shape}")
            print(f"W2 shape: {self.W2.shape}, b2 shape: {self.b2.shape}")
        
        # Layer 2: a1 @ W2 + b2
        self.z2 = self.a1 @ self.W2 + self.b2
        self.a2 = sigmoid(self.z2)
        
        if verbose:
            print(f"z2 = a1 @ W2 + b2, shape: {self.z2.shape}")
            print(f"a2 = Sigmoid(z2), shape: {self.a2.shape}")
        
        return self.a2
    
    def visualize_weights(self):
        """Visualize the weight matrices."""
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        
        # Plot W1
        im1 = axes[0].imshow(self.W1.T, cmap='RdBu', aspect='auto')
        axes[0].set_title(f'W1 (Input → Hidden)\nShape: {self.W1.shape}')
        axes[0].set_xlabel('Input Units')
        axes[0].set_ylabel('Hidden Units')
        plt.colorbar(im1, ax=axes[0])
        
        # Plot W2
        im2 = axes[1].imshow(self.W2.T, cmap='RdBu', aspect='auto')
        axes[1].set_title(f'W2 (Hidden → Output)\nShape: {self.W2.shape}')
        axes[1].set_xlabel('Hidden Units')
        axes[1].set_ylabel('Output Units')
        plt.colorbar(im2, ax=axes[1])
        
        plt.tight_layout()
        plt.show()

# Create a simple neural network
np.random.seed(42)
nn = SimpleNeuralNetwork(input_size=3, hidden_size=4, output_size=2)

# Create sample input
X_sample = np.array([[1.0, 0.5, -0.2],
                     [0.3, -1.0, 0.8],
                     [-0.5, 0.0, 1.2]])

print("=== Neural Network Forward Pass ===")
output = nn.forward(X_sample, verbose=True)
print(f"\nFinal output: {output}")

# Visualize the weight matrices
nn.visualize_weights()

# Show how different inputs activate different patterns
test_inputs = [
    np.array([[1, 0, 0]]),   # Unit vector in first dimension
    np.array([[0, 1, 0]]),   # Unit vector in second dimension
    np.array([[0, 0, 1]]),   # Unit vector in third dimension
    np.array([[1, 1, 1]]),   # All ones
    np.array([[-1, -1, -1]]) # All negative ones
]

print("\n=== Activation Patterns for Different Inputs ===")
activations = []
for i, test_input in enumerate(test_inputs):
    output = nn.forward(test_input)
    activations.append(nn.a1[0])  # Hidden layer activations
    print(f"Input {test_input[0]} → Hidden: {nn.a1[0]} → Output: {output[0]}")

# Visualize activation patterns
activations = np.array(activations)
plt.figure(figsize=(10, 6))
plt.imshow(activations.T, cmap='RdYlBu', aspect='auto')
plt.colorbar(label='Activation Level')
plt.xlabel('Input Pattern')
plt.ylabel('Hidden Unit')
plt.title('Hidden Layer Activation Patterns')
plt.xticks(range(len(test_inputs)), 
           ['[1,0,0]', '[0,1,0]', '[0,0,1]', '[1,1,1]', '[-1,-1,-1]'])
plt.show()

print("\n=== Key Insights ===")
print("1. Each layer is just a matrix multiplication followed by non-linearity")
print("2. The weight matrices encode learned transformations")
print("3. Different inputs create different activation patterns")
print("4. The hidden layer learns to represent useful features")
print("5. Linear algebra makes this all computationally efficient!")

## 🎯 Key Takeaways

Through this interactive exploration, you've seen how:

1. **Vectors represent data points and directions** in high-dimensional spaces
2. **Matrices encode transformations** that can rotate, scale, and shear
3. **Eigenvalues/eigenvectors reveal fundamental properties** of these transformations
4. **PCA finds principal directions of variance** using eigenanalysis
5. **Linear regression uses matrix operations** for closed-form solutions
6. **Neural networks are compositions** of linear transformations and non-linearities

## 🚀 Next Steps

Ready to dive deeper? Continue with:
- **[Step 2: Calculus & Gradients](02_calculus_gradients.md)** - Learn how models actually learn through optimization
- **Exercises**: Complete the [Linear Algebra Exercises](../exercises/01_linear_algebra_solutions.md)
- **Code**: Explore the [implementation details](../code/) for deeper understanding

## 📚 Additional Resources

- **3Blue1Brown**: "Essence of Linear Algebra" video series
- **Khan Academy**: Linear Algebra course
- **MIT 18.06**: Gilbert Strang's Linear Algebra lectures

---

*"Linear algebra is the foundation upon which the entire edifice of machine learning rests."*